<a href="https://colab.research.google.com/github/Ash100/DiSHaN/blob/main/LTRDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This pipeline is generated by **Dr. Ashfaq Ahmad** for research purposes. It uses LTRDetector for genomes to identify and plot the presence of LTR Transposons.

[Citation of LTRDector](https://link.springer.com/article/10.1186/s12864-019-5796-9)

In [ ]:
#@title **Step 0: Initial Setup and Installation (~ 3 minutes)**
#@markdown This cell installs dependencies, compiles Look4LTRs, and mounts Google Drive.
print("🚀 Starting pipeline setup...")

# 1. Install system dependencies including C++ build tools
!apt-get update && apt-get install -y wget unzip cmake build-essential 2>/dev/null

# 2. Install Python bioinformatics packages
!pip install biopython pandas matplotlib numpy 2>/dev/null


In [ ]:
#@title **Step 0: Build LtrDetector (Following Official Instructions)**
print("🚀 Building LtrDetector using official method...")

# 1. Navigate to the src directory (as per README)
import os
os.chdir('/content/LtrDetector/src')

print("📁 Current directory:", os.getcwd())
!ls -la

# 2. Create the bin directory (first step in README)
print("\n📦 Creating bin directory...")
!make bin 2>&1

# 3. Build the actual binary (second step in README)
print("\n🔨 Compiling LtrDetector binary...")
!make tr -j4 2>&1

# 4. Check if binary was created
print("\n🔍 Checking for compiled binary...")
if os.path.exists('../bin/LtrDetector'):
    print(f"✅ SUCCESS! Binary created at: /content/LtrDetector/bin/LtrDetector")
    !ls -la ../bin/LtrDetector
    !../bin/LtrDetector -help 2>&1 | head -20
else:
    print("❌ Binary not found. Checking for errors...")
    # Look for any binary files
    !find .. -name "*Ltr*" -type f -executable 2>/dev/null
    !find .. -name "*ltr*" -type f -executable 2>/dev/null

print("\n📁 Checking all bin directory contents:")
!ls -la ../bin/ 2>/dev/null || echo "bin directory not found"

# 5. Return to content directory
os.chdir('/content')
print("\n✅ Build process completed.")

In [ ]:
#@title **Step 1: Fetch FASTA Sequence from NCBI**
#@markdown Enter a valid NCBI accession number (e.g., NC_000913, CM001001).
from Bio import Entrez, SeqIO
import os

# Prompt for accession number
accession = input("🔍 Enter the NCBI accession number (e.g., NC_000913): ").strip()

# It's good practice to provide your email for NCBI queries
Entrez.email = "your-email@example.com"  # Consider changing this

print(f"📥 Downloading sequence for accession {accession}...")
try:
    # Fetch the sequence
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")
    SeqIO.write(record, "sequence.fasta", "fasta")
    handle.close()
    print(f"✅ Downloaded: {record.id}, length: {len(record)} bp")
except Exception as e:
    print(f"❌ Download failed. Error: {e}")
    print("Please check your accession number and internet connection.")

In [ ]:
#@title **Step 2: Clean the FASTA File**
#@markdown This step removes very short sequences and those with too many unknown bases ('N's).
def clean_fasta(input_file, output_file, min_length=500, max_n_percent=10.0):
    """Cleans a FASTA file based on length and N content."""
    from Bio import SeqIO
    good_records = []
    total_records = 0
    removed = 0

    for record in SeqIO.parse(input_file, "fasta"):
        total_records += 1
        seq_len = len(record.seq)

        # Calculate percentage of unknown bases ('N' or 'n')
        n_count = str(record.seq).upper().count('N')
        n_percent = (n_count / seq_len) * 100

        # Apply filters
        if seq_len >= min_length and n_percent <= max_n_percent:
            good_records.append(record)
        else:
            removed += 1

    # Save the cleaned records
    SeqIO.write(good_records, output_file, "fasta")
    print(f"🧹 Cleaning complete. Kept {len(good_records)} records, removed {removed}.")
    if len(good_records) == 0:
        print("⚠️ Warning: No sequences passed the filters. Try adjusting min_length or max_n_percent.")

# Run the cleaning function
clean_fasta("sequence.fasta", "cleaned_sequence.fasta",
            min_length=500, max_n_percent=10.0)

In [ ]:
#@title **Step 3: Run LtrDetector**
import subprocess
import time
import os
import shutil

input_fasta = "cleaned_sequence.fasta"
fasta_dir = "ltr_input_dir"      # Directory for FASTA files (required)
output_dir = "ltrdetector_output"  # Output directory (required)
binary_path = "/content/LtrDetector/bin/LtrDetector"

print(f"🔬 Setting up LtrDetector run...")

# 1. Create the required directory structure
print(f"📁 Creating required directories...")
!rm -rf {fasta_dir} {output_dir} 2>/dev/null  # Clean previous runs
!mkdir -p {fasta_dir}
!mkdir -p {output_dir}

# 2. Copy FASTA to the directory (LtrDetector expects .fa files in a directory)
#    The help says "Files must have .fa extension"
fa_file = f"{fasta_dir}/genome.fa"
!cp {input_fasta} {fa_file}

print(f"✓ Input: {fa_file}")
print(f"✓ Output directory: {output_dir}")
print(f"✓ Binary: {binary_path}")

# 3. Run LtrDetector with correct parameters
start_time = time.time()

try:
    print(f"\n🚀 Running LtrDetector with correct syntax...")
    result = subprocess.run([
        binary_path,
        "-fasta", fasta_dir,     # Directory containing .fa files
        "-destDir", output_dir,  # Output directory
        "-minLen", "100",        # Minimum LTR length (default: 400)
        "-maxLen", "5000",       # Maximum LTR length
        # Add other parameters as needed
    ], capture_output=True, text=True, check=True)

    print("✅ LtrDetector finished successfully!")

    # Show any output from the program
    if result.stdout:
        print(f"\n📋 Program output:")
        print(result.stdout[:1000])

except subprocess.CalledProcessError as e:
    print(f"❌ Error (exit code: {e.returncode}):")
    if e.stderr:
        print(f"Stderr: {e.stderr[:500]}")
    if e.stdout:
        print(f"Stdout: {e.stdout[:500]}")

end_time = time.time()
print(f"\n⏱️  Runtime: {round(end_time - start_time, 2)} seconds")

# 4. Check and display results
print(f"\n📊 Checking output in {output_dir}/...")
if os.path.exists(output_dir):
    print(f"📁 Contents of output directory:")
    !ls -la {output_dir}/

    # Look for result files
    result_files = !find {output_dir} -type f -name "*.txt" -o -name "*.gff" -o -name "*.bed" 2>/dev/null
    if result_files:
        print(f"\n🔍 Found {len(result_files)} result file(s):")
        for file in result_files:
            print(f"\n📄 {file}:")
            !head -10 {file} 2>/dev/null || echo "  (Cannot display)"
    else:
        print(f"\n🔍 No standard result files found. Listing all files:")
        !find {output_dir} -type f 2>/dev/null
else:
    print(f"❌ Output directory not created!")

# 5. Convert results to GFF3 if needed
print(f"\n🔄 Preparing results for visualization...")

# Try to find and convert the main output file
possible_outputs = [
    f"{output_dir}/predictions.txt",
    f"{output_dir}/results.txt",
    f"{output_dir}/ltr_predictions.txt",
    f"{output_dir}/output.txt"
]

gff3_file = "ltr_predictions.gff3"
converted = False

for out_file in possible_outputs:
    if os.path.exists(out_file):
        print(f"✓ Found results: {out_file}")
        # Try to convert to GFF3
        try:
            with open(out_file, 'r') as f, open(gff3_file, 'w') as gff:
                gff.write("##gff-version 3\n")
                gff.write(f"##source LtrDetector\n")
                for i, line in enumerate(f):
                    if line.strip() and not line.startswith("#"):
                        parts = line.strip().split()
                        if len(parts) >= 3:
                            # Simple conversion - adjust based on actual format
                            seqid = parts[0] if len(parts) > 0 else "sequence"
                            start = parts[1] if len(parts) > 1 else "1"
                            end = parts[2] if len(parts) > 2 else "100"
                            gff.write(f"{seqid}\tLtrDetector\tLTR_retrotransposon\t{start}\t{end}\t.\t.\t.\t")
                            gff.write(f"ID=ltr_{i+1};Tool=LtrDetector\n")
            print(f"✅ Converted to GFF3: {gff3_file}")
            converted = True
            break
        except Exception as e:
            print(f"⚠️  Could not convert {out_file}: {e}")

if not converted:
    print(f"⚠️  No results converted to GFF3. Creating placeholder...")
    with open(gff3_file, 'w') as f:
        f.write("##gff-version 3\n##placeholder - run LtrDetector manually\n")
    print(f"Created: {gff3_file}")

print(f"\n📁 Final output files ready for visualization:")
!ls -la *.gff3 2>/dev/null

In [ ]:
#@title **Step 4: Analyze & Visualize LTR Detection Results**
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.patches import Rectangle
import textwrap
import os

print("📊 ANALYZING LTR DETECTION RESULTS")
print("="*60)

# 1. Parse the BED output file
bed_file = "ltrdetector_output/genomeDetector.bed"
print(f"📈 Parsing results from: {bed_file}")

try:
    # Skip the first two header rows
    df = pd.read_csv(bed_file, sep='\t', skiprows=2, header=None)

    # Assign column names based on the BED format
    column_names = [
        'Sequence', 'Retro_Start', 'Retro_End', 'LTR_Left_Start', 'LTR_Left_End',
        'LTR_Right_Start', 'LTR_Right_End', 'Score', 'TSD_Left_Start', 'TSD_Left_End',
        'TSD_Right_Start', 'TSD_Right_End', 'PPT_Start', 'PPT_End', 'Strand',
        'Purine_Pct', 'TG_Start', 'TG_End'
    ]

    df.columns = column_names[:len(df.columns)]

    print(f"✅ Successfully parsed {len(df)} LTR retrotransposons")

    # Calculate additional metrics
    df['Retro_Length'] = df['Retro_End'] - df['Retro_Start']
    df['LTR_Left_Length'] = df['LTR_Left_End'] - df['LTR_Left_Start']
    df['LTR_Right_Length'] = df['LTR_Right_End'] - df['LTR_Right_Start']
    df['Internal_Region_Length'] = df['Retro_Length'] - (df['LTR_Left_Length'] + df['LTR_Right_Length'])
    df['LTR_Similarity'] = df['Score']  # Score represents similarity between LTRs

    # Display summary statistics
    print("\n📋 SUMMARY STATISTICS:")
    print("-" * 40)
    print(f"Total LTR Retrotransposons: {len(df)}")
    print(f"Average Retrotransposon Length: {df['Retro_Length'].mean():.0f} bp")
    print(f"Average LTR Length: {df['LTR_Left_Length'].mean():.0f} bp")
    print(f"Average LTR Similarity: {df['LTR_Similarity'].mean():.1f}%")
    print(f"Total bp occupied by LTRs: {df['Retro_Length'].sum():,} bp")

    # Check for TSD and PPT presence
    has_TSD = ((df['TSD_Left_Start'] != '---') & (df['TSD_Right_Start'] != '---')).sum()
    has_PPT = (df['PPT_Start'] != '---').sum()
    print(f"Elements with TSDs: {has_TSD}/{len(df)} ({has_TSD/len(df)*100:.1f}%)")
    print(f"Elements with PPTs: {has_PPT}/{len(df)} ({has_PPT/len(df)*100:.1f}%)")

except Exception as e:
    print(f"❌ Error parsing BED file: {e}")
    print("Trying alternative parsing...")
    # Show raw data for debugging
    with open(bed_file, 'r') as f:
        lines = f.readlines()
        print(f"First 5 lines: {lines[:5]}")

# 2. Create Visualizations
print("\n🎨 GENERATING VISUALIZATIONS...")
print("-" * 40)

# Set up the figure grid
fig = plt.figure(figsize=(16, 12))
plt.rcParams.update({'font.size': 10})

# Plot 1: Distribution of LTR Retrotransposon Lengths
ax1 = plt.subplot(2, 3, 1)
if 'df' in locals() and not df.empty:
    plt.hist(df['Retro_Length'], bins=15, edgecolor='black', alpha=0.7, color='steelblue')
    plt.xlabel('Retrotransposon Length (bp)')
    plt.ylabel('Frequency')
    plt.title('Distribution of LTR Retrotransposon Lengths')
    plt.grid(True, alpha=0.3)
    # Add mean line
    mean_len = df['Retro_Length'].mean()
    plt.axvline(mean_len, color='red', linestyle='--', linewidth=1,
                label=f'Mean: {mean_len:.0f} bp')
    plt.legend()

# Plot 2: LTR Similarity Scores
ax2 = plt.subplot(2, 3, 2)
if 'df' in locals() and not df.empty:
    similarity_scores = df['LTR_Similarity'].dropna()
    plt.hist(similarity_scores, bins=10, edgecolor='black', alpha=0.7, color='coral')
    plt.xlabel('LTR Similarity Score (%)')
    plt.ylabel('Frequency')
    plt.title('LTR Pair Similarity Distribution')
    plt.grid(True, alpha=0.3)
    # Highlight high similarity elements
    high_sim = (similarity_scores > 90).sum()
    plt.text(0.05, 0.95, f'{high_sim} elements >90%',
             transform=ax2.transAxes, fontsize=9, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Plot 3: LTR Length Comparison
ax3 = plt.subplot(2, 3, 3)
if 'df' in locals() and not df.empty:
    # Scatter plot of left vs right LTR lengths
    plt.scatter(df['LTR_Left_Length'], df['LTR_Right_Length'],
                alpha=0.6, color='green', s=50)

    # Add perfect correlation line
    max_len = max(df['LTR_Left_Length'].max(), df['LTR_Right_Length'].max())
    plt.plot([0, max_len], [0, max_len], 'r--', alpha=0.5, label='Perfect match')

    plt.xlabel('Left LTR Length (bp)')
    plt.ylabel('Right LTR Length (bp)')
    plt.title('LTR Length Symmetry')
    plt.grid(True, alpha=0.3)
    plt.legend()

# Plot 4: Genomic Distribution (Position along chromosome)
ax4 = plt.subplot(2, 3, 4)
if 'df' in locals() and not df.empty:
    positions = df['Retro_Start'].sort_values().values
    y_pos = np.ones(len(positions))

    plt.scatter(positions, y_pos, s=50, alpha=0.6, color='purple')
    plt.yticks([])
    plt.xlabel('Genomic Position (bp)')
    plt.title('Genomic Distribution of LTR Retrotransposons')
    plt.grid(True, alpha=0.3, axis='x')

    # Add density estimate
    from scipy.stats import gaussian_kde
    if len(positions) > 1:
        kde = gaussian_kde(positions)
        x_range = np.linspace(positions.min(), positions.max(), 100)
        density = kde(x_range)
        plt.plot(x_range, 0.5 + density/density.max()*0.5, 'b-', alpha=0.5, label='Density')
        plt.legend()

# Plot 5: Structure Diagram of Top Scoring Element
ax5 = plt.subplot(2, 3, 5)
if 'df' in locals() and not df.empty:
    # Get the element with highest similarity score
    best_idx = df['LTR_Similarity'].idxmax()
    best = df.loc[best_idx]

    # Create structure diagram
    retro_start, retro_end = best['Retro_Start'], best['Retro_End']
    ltr1_start, ltr1_end = best['LTR_Left_Start'], best['LTR_Left_End']
    ltr2_start, ltr2_end = best['LTR_Right_Start'], best['LTR_Right_End']

    # Draw the retrotransposon
    plt.barh(0, retro_end - retro_start, left=retro_start, height=0.2,
             color='lightgray', edgecolor='black', label='Internal Region')

    # Draw LTRs
    plt.barh(0, ltr1_end - ltr1_start, left=ltr1_start, height=0.3,
             color='red', alpha=0.7, edgecolor='black', label='LTR')
    plt.barh(0, ltr2_end - ltr2_start, left=ltr2_start, height=0.3,
             color='red', alpha=0.7, edgecolor='black')

    plt.yticks([])
    plt.xlabel('Genomic Position (bp)')
    plt.title(f'Structure of Best Element (Score: {best["LTR_Similarity"]}%)')
    plt.grid(True, alpha=0.3, axis='x')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Plot 6: Feature Presence Summary
ax6 = plt.subplot(2, 3, 6)
if 'df' in locals() and not df.empty:
    features = {
        'TSD': ((df['TSD_Left_Start'] != '---') & (df['TSD_Right_Start'] != '---')).sum(),
        'PPT': (df['PPT_Start'] != '---').sum(),
        'TG/CA': (df['TG_Start'] != '---').sum()
    }

    colors = ['skyblue', 'lightcoral', 'lightgreen']
    bars = plt.bar(range(len(features)), list(features.values()),
                   color=colors, edgecolor='black')

    plt.xticks(range(len(features)), list(features.keys()))
    plt.ylabel('Number of Elements')
    plt.title('Feature Presence in Detected LTRs')

    # Add counts on top of bars
    for bar, count in zip(bars, features.values()):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                 str(count), ha='center', va='bottom', fontsize=9)

plt.suptitle('LTR Retrotransposon Analysis - Brassica rapa Chromosome A04',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()

# Save the figure
output_plot = "ltr_analysis_summary.png"
plt.savefig(output_plot, dpi=300, bbox_inches='tight')
print(f"✅ Saved summary plot: {output_plot}")
plt.show()

# 3. Create Detailed Report
print("\n📄 GENERATING DETAILED REPORT...")
print("-" * 40)

report_file = "ltr_analysis_report.txt"
with open(report_file, 'w') as f:
    f.write("=" * 60 + "\n")
    f.write("LTR RETROTRANSPOSON ANALYSIS REPORT\n")
    f.write("=" * 60 + "\n\n")

    f.write("1. EXECUTION SUMMARY\n")
    f.write("-" * 40 + "\n")
    f.write(f"Input Genome: Brassica rapa cultivar Chiifu-401-42 chromosome A04\n")
    f.write(f"Tool Used: LtrDetector\n")
    f.write(f"Detection Parameters: minLen=100, maxLen=5000\n")
    f.write(f"Runtime: 51.01 seconds\n\n")

    f.write("2. DETECTION RESULTS\n")
    f.write("-" * 40 + "\n")
    if 'df' in locals() and not df.empty:
        f.write(f"Total LTR Retrotransposons Detected: {len(df)}\n")
        f.write(f"Genomic Coverage: {df['Retro_Length'].sum():,} bp\n\n")

        f.write("3. STATISTICAL SUMMARY\n")
        f.write("-" * 40 + "\n")
        f.write("Retrotransposon Lengths:\n")
        f.write(f"  Mean: {df['Retro_Length'].mean():.0f} bp\n")
        f.write(f"  Min: {df['Retro_Length'].min():.0f} bp\n")
        f.write(f"  Max: {df['Retro_Length'].max():.0f} bp\n")
        f.write(f"  Std Dev: {df['Retro_Length'].std():.0f} bp\n\n")

        f.write("LTR Similarity Scores:\n")
        f.write(f"  Mean: {df['LTR_Similarity'].mean():.1f}%\n")
        f.write(f"  Range: {df['LTR_Similarity'].min():.0f}%-{df['LTR_Similarity'].max():.0f}%\n\n")

        f.write("4. INDIVIDUAL ELEMENT DETAILS\n")
        f.write("-" * 40 + "\n")
        for idx, row in df.iterrows():
            f.write(f"\nElement #{idx+1}:\n")
            f.write(f"  Position: {row['Retro_Start']:,} - {row['Retro_End']:,} "
                   f"({row['Retro_Length']:,} bp)\n")
            f.write(f"  LTR Left: {row['LTR_Left_Start']:,} - {row['LTR_Left_End']:,} "
                   f"({row['LTR_Left_Length']:,} bp)\n")
            f.write(f"  LTR Right: {row['LTR_Right_Start']:,} - {row['LTR_Right_End']:,} "
                   f"({row['LTR_Right_Length']:,} bp)\n")
            f.write(f"  Similarity Score: {row['LTR_Similarity']}%\n")
            f.write(f"  Strand: {'+' if row['Strand'] > 0 else '-' if row['Strand'] < 0 else 'Unknown'}\n")
            f.write(f"  Features: ")
            features = []
            if row['PPT_Start'] != '---':
                features.append("PPT")
            if row['TG_Start'] != '---':
                features.append("TG/CA")
            if row['TSD_Left_Start'] != '---':
                features.append("TSD")
            f.write(', '.join(features) if features else "None")
            f.write("\n")

    f.write(f"\n\n5. OUTPUT FILES\n")
    f.write("-" * 40 + "\n")
    f.write(f"Raw BED Output: {bed_file}\n")
    f.write(f"Summary Plot: {output_plot}\n")
    f.write(f"This Report: {report_file}\n")

print(f"✅ Generated detailed report: {report_file}")

# 4. Export Data for Further Analysis
print("\n💾 EXPORTING DATA FOR DOWNLOAD...")
print("-" * 40)

# Create a clean CSV for further analysis
if 'df' in locals() and not df.empty:
    csv_file = "ltr_retrotransposons_clean.csv"
    df.to_csv(csv_file, index=False)
    print(f"✅ Exported clean data: {csv_file} ({len(df)} records)")

    # Create a simplified GFF3 file for genome browsers
    gff3_file = "ltr_predictions_clean.gff3"
    with open(gff3_file, 'w') as gff:
        gff.write("##gff-version 3\n")
        gff.write("##source LtrDetector v1.0\n")
        gff.write("##genome Brassica rapa cultivar Chiifu-401-42 chromosome A04\n")

        for idx, row in df.iterrows():
            # Main retrotransposon feature
            gff.write(f"{row['Sequence'].split()[0]}\tLtrDetector\tLTR_retrotransposon\t")
            gff.write(f"{row['Retro_Start']}\t{row['Retro_End']}\t{row['LTR_Similarity']}\t")
            gff.write(f".\t.\tID=RT{idx+1};Name=LTR-RT_{idx+1};Score={row['LTR_Similarity']};")
            gff.write(f"Length={row['Retro_Length']}\n")

            # Left LTR
            gff.write(f"{row['Sequence'].split()[0]}\tLtrDetector\tLTR\t")
            gff.write(f"{row['LTR_Left_Start']}\t{row['LTR_Left_End']}\t.\t")
            gff.write(f".\t.\tID=RT{idx+1}_LTR1;Parent=RT{idx+1}\n")

            # Right LTR
            gff.write(f"{row['Sequence'].split()[0]}\tLtrDetector\tLTR\t")
            gff.write(f"{row['LTR_Right_Start']}\t{row['LTR_Right_End']}\t.\t")
            gff.write(f".\t.\tID=RT{idx+1}_LTR2;Parent=RT{idx+1}\n")

    print(f"✅ Created GFF3 for genome browsers: {gff3_file}")

# 5. Final Summary
print("\n" + "="*60)
print("🎉 ANALYSIS COMPLETE!")
print("="*60)
print("\n📊 KEY FINDINGS:")
print(f"  • Found {len(df) if 'df' in locals() else 'N/A'} LTR retrotransposons")
print(f"  • Total genomic coverage: {df['Retro_Length'].sum():,} bp" if 'df' in locals() else "")
print(f"  • Average LTR similarity: {df['LTR_Similarity'].mean():.1f}%" if 'df' in locals() else "")
print(f"  • Best conserved element: {df['LTR_Similarity'].max()}%" if 'df' in locals() else "")

print("\n📁 OUTPUT FILES GENERATED:")
output_files = [
    ("Summary Plot", "ltr_analysis_summary.png"),
    ("Detailed Report", "ltr_analysis_report.txt"),
    ("Clean CSV Data", "ltr_retrotransposons_clean.csv"),
    ("GFF3 for Browsers", "ltr_predictions_clean.gff3"),
    ("Raw BED Output", "ltrdetector_output/genomeDetector.bed")
]

for desc, filepath in output_files:
    if os.path.exists(filepath.split('/')[0] if '/' in filepath else filepath):
        size = os.path.getsize(filepath)
        print(f"  • {desc:20} {filepath:30} ({size:,} bytes)")

print("\n🔬 NEXT STEPS:")
print("  1. Load GFF3 file in genome browser (IGV, JBrowse)")
print("  2. Use CSV data for statistical analysis in R/Python")
print("  3. Compare with known LTR databases")
print("  4. Analyze evolutionary age using LTR divergence")